In [3]:
import pandas as pd
import os
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus

In [4]:
df_raw = pd.read_csv('../data/6548.csv', sep=';', thousands='.', decimal=',')

In [5]:
df_raw.head()

,Comunidades y Ciudades Autónomas,Sexo,Edad,Periodo,Total
0,Total,Total,Menores de un año,2023,848.0
1,Total,Total,Menores de un año,2022,869.0
2,Total,Total,Menores de un año,2021,861.0
3,Total,Total,Menores de un año,2020,890.0
4,Total,Total,Menores de un año,2019,977.0


In [6]:
print(df_raw.dtypes)

Comunidades y Ciudades Autónomas     object
Sexo                                 object
Edad                                 object
Periodo                               int64
Total                               float64
dtype: object


In [7]:
age_map = {
    'Menores de un año': '0',
    '100 y más años': '100'
}
df_raw['age'] = (
    df_raw['Edad']
      .replace(age_map)           
      .str.extract(r'(\d+)')      
      .astype(int)         
)

In [8]:
# 2. Mapear sexo a T/M/F
sex_map = {
    'Total':   'T',
    'Hombres': 'M',
    'Mujeres': 'F'
}
df_raw['sex'] = df_raw['Sexo'].map(sex_map).astype('category')

In [9]:
# Procesar la columna de CCAA
#    - "Total"       -> code=None, name=None
#    - "No residente" -> descartarla
#    - "<CC> <Nombre>" -> extraer código de 2 chars y nombre

geo = df_raw['Comunidades y Ciudades Autónomas']
mask_total = geo == 'Total'
mask_nores = geo == 'No residente'

# Incluir Total como vacíos
df_raw.loc[mask_total, 'code_ine_ccaa'] = None
df_raw.loc[mask_total, 'name_ccaa'] = None

# Eliminar "No residente"
df_raw = df_raw[~mask_nores]

In [10]:
df_raw.sample()

,Comunidades y Ciudades Autónomas,Sexo,Edad,Periodo,Total,age,sex,code_ine_ccaa,name_ccaa
170725,11 Extremadura,Hombres,50 años,2014,32.0,50,M,NaN,NaN


In [11]:
# El resto: separar en código (2 chars) y nombre
rest = df_raw['Comunidades y Ciudades Autónomas'].notna() & \
       (df_raw['Comunidades y Ciudades Autónomas'] != 'Total')

codes_names = df_raw.loc[rest, 'Comunidades y Ciudades Autónomas'] \
                   .str.split(pat=' ', n=1, expand=True)

df_raw.loc[rest, 'code_ine_ccaa'] = codes_names[0]
df_raw.loc[rest, 'name_ccaa']    = codes_names[1]

# Renombrar y quedarte sólo con las columnas finales de defunciones
df_final = df_raw.rename(columns={
    'Periodo':  'year',
    'age':   'age',
    'sex':   'sex',
    'Total': 'deaths'
})[['year','age','sex','deaths','code_ine_ccaa','name_ccaa']]

# Verificación
print(df_final.dtypes)
print(df_final.head())

year                int64
age                 int64
sex              category
deaths            float64
code_ine_ccaa      object
name_ccaa          object
dtype: object
   year  age sex  deaths code_ine_ccaa name_ccaa
0  2023    0   T   848.0          None      None
1  2022    0   T   869.0          None      None
2  2021    0   T   861.0          None      None
3  2020    0   T   890.0          None      None
4  2019    0   T   977.0          None      None


In [14]:
df_final.sample()

,year,age,sex,deaths,code_ine_ccaa,name_ccaa
166966,2000,74,T,282.0,11,Extremadura


In [13]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]

In [15]:
coll = db['INE_6548_POBLACION_DEFUNCIONES_RAW']
coll.drop()
coll.insert_many(df_final.to_dict('records'))
print("Registros en la colección: ", coll.count_documents({}))
print("Número de filas de df_raw: ", len(df_final))

Registros en la colección:  296940
Número de filas de df_raw:  296940
